In [1]:
import pandas as pd
from gurobipy import *

xls = pd.ExcelFile('Proyecto_1_TransporteMaritimo_Dataset1.xlsx', engine='openpyxl')
nombres= xls.sheet_names
df1 = pd.read_excel(xls, nombres[0])
df2 = pd.read_excel(xls, nombres[1])
df3= pd.read_excel(xls, nombres[2])
df4= pd.read_excel(xls, nombres[3])
df5= pd.read_excel(xls, nombres[4])
#df6= pd.read_excel(xls, nombres[5])

In [2]:
n_cargos = len(df3)
n_cargos

15

In [3]:
V= list(df1["ID_Barco"]) ## conjunto barcos
n_barcos = len(V)
print(V)

[1, 2, 3, 4]


In [4]:
Kv= {} # conjunto capacidades de barcos
for i in range(0,len(V)):
    Kv[i+1]= list(df1["Capacidad"])[i]
Kv

{1: 16500, 2: 5800, 3: 3570, 4: 3570}

In [5]:
# N^P
nodos_carga= list(df3["ID_Cargo"])
print(nodos_carga)

#N^D
nodos_descarga= [i+len(nodos_carga) for i in range(1,len(nodos_carga)+1)] ##conjunto nodos de descarga
print(nodos_descarga)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [6]:
cargas = df3[["ID_Cargo","ID_Puerto_Origen", "Tamano", "Costo_SPOT","LT_Carga","RT_Carga"]].rename(columns={"ID_Puerto_Origen": "ID_Puerto ", "LT_Carga":"LT","RT_Carga":"RT"})
descargas = df3[["ID_Cargo","ID_Puerto_Destino", "Tamano", "Costo_SPOT","LT_Descarga","RT_Descarga"]].rename(columns={"ID_Puerto_Destino": "ID_Puerto ", "LT_Descarga":"LT","RT_Descarga":"RT"})
nodos = pd.concat([cargas,descargas],axis=0)
nodos = nodos.reset_index()[["ID_Cargo","ID_Puerto ", "Tamano", "Costo_SPOT","LT","RT"]]
nodos.index+=1
N = nodos.to_dict("index")

puertos_barco = df1["ID_Puerto "]
t_origen = df1["Tiempo_Inicio"].to_dict()
o = {}
d = {}
for i in range(len(puertos_barco.to_dict())):
  o[i+1] = i+1+n_cargos*2 
  N[i+1+n_cargos*2] = {'Costo_SPOT': 0,
  'ID_Cargo': 0,
  'ID_Puerto ': puertos_barco.to_dict()[i],
  'LT': t_origen[i],
  'RT': t_origen[i],
  'Tamano': 0}
  d[i+1]=i+1+n_cargos*2+len(puertos_barco.to_dict())
  N[i+1+n_cargos*2+len(puertos_barco.to_dict())]={'Costo_SPOT': 0,
  'ID_Cargo': 0,
  'ID_Puerto ': 0,
  'LT':1388,
  'RT': 1388,
  'Tamano': 0}
nodos_index = [i for i in N.keys()]

print(o) # nodos de origen
print(d) # nodos de destino
print(N) # todos los nodos con su información, por ejemplo el nodo 1: {'ID_Cargo': 1, 'ID_Puerto ': 25, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 455}

{1: 31, 2: 32, 3: 33, 4: 34}
{1: 35, 2: 36, 3: 37, 4: 38}
{1: {'ID_Cargo': 1, 'ID_Puerto ': 25, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 455}, 2: {'ID_Cargo': 2, 'ID_Puerto ': 9, 'Tamano': 6919, 'Costo_SPOT': 536901, 'LT': 149, 'RT': 221}, 3: {'ID_Cargo': 3, 'ID_Puerto ': 7, 'Tamano': 3395, 'Costo_SPOT': 474427, 'LT': 5, 'RT': 77}, 4: {'ID_Cargo': 4, 'ID_Puerto ': 6, 'Tamano': 5688, 'Costo_SPOT': 458905, 'LT': 244, 'RT': 316}, 5: {'ID_Cargo': 5, 'ID_Puerto ': 38, 'Tamano': 3419, 'Costo_SPOT': 644852, 'LT': 338, 'RT': 410}, 6: {'ID_Cargo': 6, 'ID_Puerto ': 10, 'Tamano': 7277, 'Costo_SPOT': 733327, 'LT': 127, 'RT': 199}, 7: {'ID_Cargo': 7, 'ID_Puerto ': 26, 'Tamano': 2733, 'Costo_SPOT': 276767, 'LT': 621, 'RT': 693}, 8: {'ID_Cargo': 8, 'ID_Puerto ': 14, 'Tamano': 6074, 'Costo_SPOT': 1037825, 'LT': 238, 'RT': 310}, 9: {'ID_Cargo': 9, 'ID_Puerto ': 5, 'Tamano': 2714, 'Costo_SPOT': 326529, 'LT': 39, 'RT': 111}, 10: {'ID_Cargo': 10, 'ID_Puerto ': 9, 'Tamano': 1497, 'Costo_SPOT'

In [7]:
compatibilidad = {}
for i in V:
    compatibilidad[i] = [int(j) for j in list(df2.loc[i-1])[1:] if j>0]
compatibilidad

Nv = {}
for i in V:
    Nv[i] = []
    for j in N.keys():
        if N[j]["ID_Cargo"] in compatibilidad[i] or (j == i+n_cargos*2 or j==i+n_cargos*2+len(V)): 
            Nv[i].append(j)
Nv

{1: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  10,
  11,
  12,
  13,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  25,
  26,
  27,
  28,
  30,
  31,
  35],
 2: [1,
  3,
  4,
  5,
  7,
  9,
  10,
  11,
  12,
  14,
  15,
  16,
  18,
  19,
  20,
  22,
  24,
  25,
  26,
  27,
  29,
  30,
  32,
  36],
 3: [1, 3, 5, 7, 9, 10, 12, 16, 18, 20, 22, 24, 25, 27, 33, 37],
 4: [1, 5, 7, 9, 10, 12, 14, 16, 20, 22, 24, 25, 27, 29, 34, 38]}

In [8]:
"""import itertools
Av = {}
for v in V:
    Av[v] = list(itertools.permutations(Nv[v],2))
"""
Av={}
indices = []
for v in V:
    Av[v] = [] 
    for i in Nv[v]:
        for j in Nv[v]:
                Av[v].append((i,j))
                indices.append((i,j,v))
indices = tuplelist(indices)
print(Av[1])

[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 10), (1, 11), (1, 12), (1, 13), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 25), (1, 26), (1, 27), (1, 28), (1, 30), (1, 31), (1, 35), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 10), (2, 11), (2, 12), (2, 13), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 25), (2, 26), (2, 27), (2, 28), (2, 30), (2, 31), (2, 35), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 10), (3, 11), (3, 12), (3, 13), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21), (3, 22), (3, 23), (3, 25), (3, 26), (3, 27), (3, 28), (3, 30), (3, 31), (3, 35), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 10), (4, 11), (4, 12), (4, 13), (4, 15), (4, 16), (4, 17), (4, 18), (4, 19), (4, 20), (4, 21), (4, 22), (4, 23), (4, 25), (4, 26), (4, 27), (4, 28), (4, 30), (4, 31), (4, 35), (5, 1), (5, 2), (5, 3),

In [9]:
Av[1]

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 25),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 30),
 (1, 31),
 (1, 35),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 15),
 (2, 16),
 (2, 17),
 (2, 18),
 (2, 19),
 (2, 20),
 (2, 21),
 (2, 22),
 (2, 23),
 (2, 25),
 (2, 26),
 (2, 27),
 (2, 28),
 (2, 30),
 (2, 31),
 (2, 35),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 10),
 (3, 11),
 (3, 12),
 (3, 13),
 (3, 15),
 (3, 16),
 (3, 17),
 (3, 18),
 (3, 19),
 (3, 20),
 (3, 21),
 (3, 22),
 (3, 23),
 (3, 25),
 (3, 26),
 (3, 27),
 (3, 28),
 (3, 30),
 (3, 31),
 (3, 35),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 10),
 (4, 11),
 (4, 12),
 (4, 13),
 (4, 15),
 (4, 16),
 (4, 17),
 (4, 18),
 (4, 19),
 (4, 20),
 (4, 21),
 (

In [10]:
NP_v = {}
for v in V:
    NP_v[v] = [i for i in nodos_carga if i in Nv[v]]

ND_v = {}
for v in V:
    ND_v[v] = [i for i in nodos_descarga if i in Nv[v]]

print(NP_v)
print(ND_v)

{1: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15], 2: [1, 3, 4, 5, 7, 9, 10, 11, 12, 14, 15], 3: [1, 3, 5, 7, 9, 10, 12], 4: [1, 5, 7, 9, 10, 12, 14]}
{1: [16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30], 2: [16, 18, 19, 20, 22, 24, 25, 26, 27, 29, 30], 3: [16, 18, 20, 22, 24, 25, 27], 4: [16, 20, 22, 24, 25, 27, 29]}


In [11]:
LT_i = {i:N[i]["LT"] for i in N.keys()} #lower bounds ventana de tiempo por nodo
RT_i = {i:N[i]["RT"] for i in N.keys()} #upper bounds ventana de tiempo por nodo

print(LT_i)
print(RT_i)

{1: 383, 2: 149, 3: 5, 4: 244, 5: 338, 6: 127, 7: 621, 8: 238, 9: 39, 10: 698, 11: 94, 12: 572, 13: 842, 14: 100, 15: 325, 16: 383, 17: 149, 18: 5, 19: 244, 20: 338, 21: 127, 22: 621, 23: 238, 24: 39, 25: 698, 26: 94, 27: 572, 28: 842, 29: 100, 30: 325, 31: 69, 35: 1388, 32: 0, 36: 1388, 33: 0, 37: 1388, 34: 0, 38: 1388}
{1: 455, 2: 221, 3: 77, 4: 316, 5: 410, 6: 199, 7: 693, 8: 310, 9: 111, 10: 770, 11: 166, 12: 644, 13: 914, 14: 172, 15: 397, 16: 777, 17: 588, 18: 489, 19: 624, 20: 871, 21: 657, 22: 982, 23: 967, 24: 501, 25: 1198, 26: 497, 27: 1093, 28: 1388, 29: 800, 30: 746, 31: 69, 35: 1388, 32: 0, 36: 1388, 33: 0, 37: 1388, 34: 0, 38: 1388}


In [12]:
T_ijv={} # tiempo de entre i y j (tiempo de viaje + tiempo de operación en i)
for i in N.keys():
    T_ijv[i] = {} 
    for j in N.keys():
        T_ijv[i][j] = {} 
        for v in V:
            cargo_i= N[i]["ID_Cargo"]
            cargo_j = N[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                T_ijv[i][j][v] = 0
            else: 
                puerto_origen = N[i]["ID_Puerto "]
                puerto_destino = N[j]["ID_Puerto "]
                T_ijv[i][j][v] = df4.loc[(df4["Puerto_Origen"]==puerto_origen)&(df4["Puerto_Destino"]==puerto_destino)&(df4["ID_Barco"]==v),["Tiempo_Viaje_hrs"]]["Tiempo_Viaje_hrs"].item()
                if 0<i<=n_cargos and (i,j) in Av[v]: 
                    T_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Tiempo_Origen_hrs"]]["Tiempo_Origen_hrs"].item()
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    T_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Tiempo_Destino_hrs"]]["Tiempo_Destino_hrs"].item()
                
                # se les asigna tiempo de operación igual a 0 a los (i,j) que no pertenecen a Av
                # los (i,j) que no pertenecen a Av aún tienen el tiempo entre el puerto de i y j

In [13]:
C_ijv={} # costo de entre i y j (costo de viaje + costo de operación en i)
for i in N.keys():
    C_ijv[i] = {} 
    for j in N.keys():
        C_ijv[i][j] = {} 
        for v in V:
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                C_ijv[i][j][v] = 0
            else: 
                puerto_origen = N[i]["ID_Puerto "]
                puerto_destino = N[j]["ID_Puerto "]
                C_ijv[i][j][v] = df4.loc[(df4["Puerto_Origen"]==puerto_origen)&(df4["Puerto_Destino"]==puerto_destino)&(df4["ID_Barco"]==v),["Costo_Viaje_libras"]]["Costo_Viaje_libras"].item()
                if i==11 and j==2 and i==1:
                    print(C_ijv[i][j][v])
                if 0<i<=n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Costos_Origen_hrs"]]["Costos_Origen_hrs"].item()

                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Costos_Destino_hrs"]]["Costos_Destino_hrs"].item()

            if (i,j) not in Av[v] or i==j:
                C_ijv[i][j][v] += 100000000000
                # se les asigna costo de operación igual a 0 a los (i,j) que no pertenecen a Av
                # los (i,j) que no pertenecen a Av aún tienen el costo entre el puerto de i y j

In [14]:
Qj={i:N[i]["Tamano"] for i in N.keys()}
print("Qj: ",Qj)

print("\n")

Cs = df3["Costo_SPOT"].to_dict()
print("Cs", Cs)

Qj:  {1: 2368, 2: 6919, 3: 3395, 4: 5688, 5: 3419, 6: 7277, 7: 2733, 8: 6074, 9: 2714, 10: 1497, 11: 4813, 12: 2370, 13: 7064, 14: 2308, 15: 4730, 16: 2368, 17: 6919, 18: 3395, 19: 5688, 20: 3419, 21: 7277, 22: 2733, 23: 6074, 24: 2714, 25: 1497, 26: 4813, 27: 2370, 28: 7064, 29: 2308, 30: 4730, 31: 0, 35: 0, 32: 0, 36: 0, 33: 0, 37: 0, 34: 0, 38: 0}


Cs {0: 354894, 1: 536901, 2: 474427, 3: 458905, 4: 644852, 5: 733327, 6: 276767, 7: 1037825, 8: 326529, 9: 420165, 10: 461936, 11: 547778, 12: 893561, 13: 844864, 14: 306945}


In [15]:

# Model
m = Model("Transporte Maritimo")

# variables

#x = m.addVars(indices, vtype=GRB.BINARY, name="Arco")
x = m.addVars(nodos_index, nodos_index, V, vtype=GRB.BINARY, name="Arco")
y = m.addVars(nodos_carga,vtype=GRB.BINARY,name="Cargo_Spot")
t = m.addVars(nodos_index,V,vtype=GRB.INTEGER, name="tiempo")
l = m.addVars(nodos_index,V,vtype=GRB.INTEGER,name="Inventario")


--------------------------------------------
--------------------------------------------

Using license file C:\Users\joval\gurobi.lic
Academic license - for non-commercial use only


In [16]:
# Objective function
#m.setObjective(sum(sum(C_ijv[i][j][v]*x[i,j,v] for (i,j) in Av[v]) for v in V)+sum(Cs[i-1]*y[i] for i in nodos_carga), GRB.MINIMIZE)
m.setObjective(sum(sum(C_ijv[i][j][v]*x[i,j,v] for i in nodos_index for j in nodos_index ) for v in V)+sum(Cs[i-1]*y[i] for i in nodos_carga), GRB.MINIMIZE)

In [17]:
m.addConstrs(((sum(sum(x[i,j,v] for j in Nv[v]) for v in V)  + y[i]== 1) for i in nodos_carga), name = "Const_1")
#m.addConstrs(((sum(sum(x[i,j,v] + y[i] for j in Nv[v] if (i,j) in Av[v]) for v in V) == 1) for i in nodos_carga), name = "Const_1")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>}

In [18]:
m.addConstrs(((sum(x[o[v],j,v] for j in Nv[v]) == 1) for v in V), name = "Const_2")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [19]:
m.addConstrs((((sum(x[i,j,v] for j in Nv[v])-sum(x[j,i,v] for j in Nv[v]))==0) for v in V for i in Nv[v] if i<=n_cargos*2), name = "Const_3")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Mode

In [20]:
m.addConstrs(((sum(x[j,d[v],v] for j in Nv[v]) == 1) for v in V), name = "Const_4")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [21]:
m.addConstrs(((l[i,v]+Qj[j]-l[j,v]<=Kv[v]*(1-x[i,j,v])) for v in V  for j in NP_v[v] for i in Nv[v] if ((i,j) in Av[v])), name = "Const_5")

{(1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 21): <gurobi.Constr *Awaitin

In [22]:
m.addConstrs(((l[i,v]-Qj[j]-l[n_cargos+j,v]<=Kv[v]*(1-x[i,n_cargos+j,v]))  for v in V for j in NP_v[v] for i in Nv[v] if ((i,j+n_cargos) in Av[v])), name = "Const_6")

{(1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 21): <gurobi.Constr *Awaitin

In [23]:
m.addConstrs(((l[i,v]<=Kv[v]) for v in V for i in NP_v[v]), name = "Const_7_ub")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Upda

In [24]:
m.addConstrs(((l[i,v]>=0) for v in V for i in NP_v[v]), name = "Const_7_lb")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Upda

In [25]:
m.addConstrs(((t[i,v]+T_ijv[i][j][v]-t[j,v]<=(RT_i[i]+T_ijv[i][j][v])*(1-x[i,j,v])) for v in V for (i,j) in Av[v]), name = "Const_8")

{(1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 21): <gurobi.Constr *Awaitin

In [26]:
m.addConstrs(((sum(x[i,j,v] for j in Nv[v])-sum(x[n_cargos+i,j,v] for j in Nv[v])==0) for v in V for i in NP_v[v]), name = "Const_9")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Upda

In [27]:

m.addConstrs(((t[i,v]+T_ijv[i][n_cargos+i][v]-t[n_cargos+i,v]<=0) for v in V for i in NP_v[v] ), name = "Const_10")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Upda

In [28]:
m.addConstrs(((t[i,v]<=RT_i[i]) for v in V for i in Nv[v]), name = "Const_11_ub")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Mode

In [29]:
m.addConstrs(((LT_i[i]<=t[i,v]) for v in V for i in Nv[v]), name = "Const_11_lb")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Mode

In [30]:
# Solve
m.optimize()

valorobjetivo = m.objval

"""for v in m.getVars():
    print('%s %g' % (v.varName, v.x))"""

print('\n Valor óptimo: %8.4f \n' % valorobjetivo)

m.write("Transporte_final.sol")
m.write("model.lp")

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3995 rows, 6095 columns and 17125 nonzeros
Model fingerprint: 0xf11bd41a
Variable types: 0 continuous, 6095 integer (5791 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [9e+02, 1e+11]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.600003e+12
Presolve removed 2688 rows and 5210 columns
Presolve time: 0.13s
Presolved: 1307 rows, 885 columns, 5666 nonzeros
Found heuristic solution: objective 6.000083e+11
Variable types: 0 continuous, 885 integer (770 binary)

Root relaxation: objective 1.627596e+06, 524 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1627595.99    0  104 6

In [31]:
#x[31,31,1].Xn

In [32]:
for v in V:
    for i in nodos_index:
        for j in nodos_index:
            if x[i,j,v].X==1:
                print(f"x[{i},{j},{v}] == 1")

x[1,16,1] == 1
x[2,17,1] == 1
x[8,26,1] == 1
x[11,2,1] == 1
x[13,28,1] == 1
x[16,23,1] == 1
x[17,8,1] == 1
x[23,13,1] == 1
x[26,1,1] == 1
x[28,35,1] == 1
x[31,11,1] == 1
x[4,19,2] == 1
x[7,22,2] == 1
x[9,24,2] == 1
x[15,30,2] == 1
x[19,15,2] == 1
x[22,36,2] == 1
x[24,4,2] == 1
x[30,7,2] == 1
x[32,9,2] == 1
x[3,18,3] == 1
x[5,20,3] == 1
x[12,27,3] == 1
x[18,5,3] == 1
x[20,12,3] == 1
x[27,37,3] == 1
x[33,3,3] == 1
x[10,25,4] == 1
x[14,29,4] == 1
x[25,38,4] == 1
x[29,10,4] == 1
x[34,14,4] == 1


In [33]:
l[28,1].X

9436.0

In [34]:
t[28,1].X

1388.0

In [35]:
s = 25
v = 4
print(LT_i[s])
print(t[s,v].X)
print(RT_i[s])

print(l[s,v].X)

698
1104.0
1198
2073.0


In [36]:
Qj[8]

6074